In [190]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['C:\\Users\\troy\\Documents\\Code\\Libraries', 'C:\\Users\\troy\\Documents\\Excel Sheets', 'C:/Users/troy/Documents/Code/Libraries'])
from datetime import date, datetime, timedelta
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
import altair as alt

%load_ext autoreload
%autoreload 2
%matplotlib qt

from FinAPI.MacroData import Inflation as cpi_api
import pandas as pd
import numpy as np

alt.renderers.enable('default')

Python 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)] on win32
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


RendererRegistry.enable('default')

## Good vs. Commodity Prices

In [3]:
dataset = cpi_api.ExogenousVariables.energy_commodity_indices
df = cpi_api.load_historical_data_dict(dataset)
df = df.fillna(method='FFILL')
df = df.resample('1M').last()


cpi_df = cpi_api.load_historical_data_dict(cpi_api.DataSet.cpi_index)
cpi_cur_nondur_df = cpi_api.load_historical_data_dict(cpi_api.DataSet.cpi_good_dur_nondur)
cpi_good_serv_df = cpi_api.load_historical_data_dict(cpi_api.DataSet.cpi_good_serv_indexes)
cpi_energy_df = cpi_api.load_historical_data_dict(cpi_api.DataSet.cpi_energy_index)

# Subsets based on the correlation
INDUSTRIAL_SUBSET = ['S&P Industrial', 'FIBER Steel', 'CRB Spot Commod', 'FIBER US Commd', 'FIBER Metals']
ENERGY_SUBSET =['S&P Energy Spot', 'US Avg Gasoline', 'US Brent Spot Avg', 'FIBER Petroleum', 'FIBER CrudOil', 'S&P GSCI', 'Bloomberg Commd']
MISC = ['US electricity all sectors', 'US Natural Gas Spot']

# Monthly Changes
df_chg = df.pct_change(1).dropna()
cpi_chg = cpi_df.pct_change(1).dropna()
cpi_dur_nondur_chg = cpi_cur_nondur_df.pct_change(1).dropna()
cpi_good_serv_chg = cpi_good_serv_df.pct_change(1).dropna()
cpi_energy_chg = cpi_energy_df.pct_change(1).dropna()


In [25]:
# Check the correlatin between the commodity indices and the Cpi indices
import pandas as pd
corr_dict = {}
df_latest = df_chg.iloc[-120:, :]
for col in cpi_chg:
    corr_dict[col] = df_latest.corrwith(cpi_chg[col])
for col in cpi_dur_nondur_chg:
    corr_dict[col] = df_latest.corrwith(cpi_dur_nondur_chg[col])
for col in cpi_good_serv_chg:
    corr_dict[col] = df_latest.corrwith(cpi_good_serv_chg[col])
for col in cpi_energy_chg:
    corr_dict[col] = df_latest.corrwith(cpi_energy_chg[col])

cpi_common_corr = pd.DataFrame(corr_dict)
cpi_common_corr = cpi_common_corr.loc[INDUSTRIAL_SUBSET + ENERGY_SUBSET + MISC]

# alt.Chart(cpi_common_corr).mark_rect().encode(x='')
melt_df = cpi_common_corr.melt(var_name="CPI category", ignore_index=False).reset_index()
alt.Chart(melt_df).mark_rect().encode(x='CPI category:N', y='index:N', color='value:Q')

alt.Chart(...)

## Run a ARD Regression to automatically select relevant variables

In [ ]:
import statsmodels.graphics.tsaplots

#### Data Definition

The exogenous market variables are split into test and train and the model is trained. 

I have left out the the period form Aug21 -> Aug22 as test, because this has been an unusual CPI period with 40y high inflation prints

The commented out part is lagged version of the exogenous variables, where the varaibles are lagged by the amount specified in the the lag_df dataframe above. I found no improvement in the prediction ability of the model.

In [228]:
endog = cpi_chg.iloc[-120:-12, :]['EnergyCPI']
outsample = cpi_chg.iloc[-12:, :]['EnergyCPI']

regressor = df_chg.copy()

# Pure exogenous + contemporous model
exog = regressor.loc[endog.index, :]
exog_out = regressor.loc[outsample.index, :]


##### Alternative 1. Creting Current + Lagged Feature set using Cross Correlation to determine the best lag

In [225]:
# Correaltion taking into account lead and lag

def lagged_max_corr(s_dep, s_indep):
    c_max = -1.
    lag_max = 0
    for shift in range(0, 13):
        t = s_indep.shift(shift).dropna()
        x_sub = t[t.index.intersection(s_dep.index)]
        y_sub = s_dep[t.index.intersection(s_dep.index)]
        c = x_sub.corr(y_sub)
        if c > c_max:
            c_max = c
            lag_max = shift
    return c_max, lag_max

df_latest = df_chg.iloc[-132:, :]

corr_df_dict = dict()
lag_df_dict = dict()

for c in df_latest:
    corr_dict = dict()
    lag_dict = dict()
    for dataset in  [cpi_chg, cpi_dur_nondur_chg, cpi_good_serv_chg, cpi_energy_chg]:
        for col in dataset:
            corr, lag = lagged_max_corr(dataset[col], df_latest[c])
            corr_dict[col] = corr
            lag_dict[col] = lag
    corr_df_dict[c] = pd.Series(corr_dict)
    lag_df_dict[c] = pd.Series(lag_dict)

corr_df = pd.DataFrame(corr_df_dict).T
lag_df = pd.DataFrame(lag_df_dict).T

# Apply pre calclculated lags
regressor = df_chg.copy()

lags = lag_df['EnergyCPI']
for c in regressor:
    lag = lags[c]
    regressor[c] = regressor[c].shift(lag)

# Pure exogenous + contemporous model
exog = regressor.loc[endog.index, :]
exog_out = regressor.loc[outsample.index, :]

In [221]:
corr_df = corr_df.loc[INDUSTRIAL_SUBSET + ENERGY_SUBSET + MISC]

# alt.Chart(cpi_common_corr).mark_rect().encode(x='')
melt_df = corr_df.melt(var_name="CPI category", ignore_index=False).reset_index()
alt.Chart(melt_df).mark_rect().encode(x='CPI category:N', y='index:N', color='value:Q')

alt.Chart(...)

##### Alternative 2: Creating Current + Lagged Feature set using Xgboost to compute feature importance

This did not improve the forecasting ability of the model. This produced good insample fit but poor outsample fit.

In [217]:
from xgboost import XGBRegressor

# Apply all lags upto 12 months and then leave it to the algorithm to figure out
regressor = df_chg.copy()
for c in regressor:
    for lag in range(1, 13):
        regressor["%s - %s" % (c, lag)] = regressor[c].shift(lag)
regressor = regressor.copy()
print("exhaustive list of regressor done ...")

# Pure exogenous + contemporous model
exog = regressor.loc[endog.index, :]
exog_out = regressor.loc[outsample.index, :]

# determin the relevant columns
print("Running xgbboost ...")
model = XGBRegressor()
model.fit(exog, endog)
feature_importance = model.feature_importances_
ind = np.argpartition(feature_importance, -20)[-20:]
relevant_cols = exog.columns[ind]
rel_col_weights = feature_importance[ind]
print("relevant columns determined ...")
plt.bar(relevant_cols, rel_col_weights)

# Select the cols and define exogenous variables
regressor = regressor[relevant_cols]

exog = regressor.loc[endog.index, :]
exog_out = regressor.loc[outsample.index, :]

exhaustive list of regressor done ...
Running xgbboost ...
relevant columns determined ...


C:\Users\troy\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  import sys


#### ARD Regression

In [229]:
from sklearn.linear_model import ARDRegression

ard = ARDRegression(compute_score=True, n_iter=30, fit_intercept=True).fit(exog, endog.values)
coef_series = pd.Series(ard.coef_, exog.columns)

y_ard, y_ard_std = ard.predict(exog.values, return_std=True)
resid = endog.values - y_ard

data_df = pd.DataFrame(dict(dates=endog.index, 
                            energy_cpi=endog, 
                            fitted=pd.Series(y_ard, endog.index), 
                            resid=pd.Series(resid, endog.index)))

print("In sample Rsq of the model is - %s" % np.round(ard.score(y=endog.values, X=exog), 2))
print("Out Sample Rsq of the model is - %s" % np.round(ard.score(y=outsample.values, X=exog_out), 2))
print("Coefs Selected ")
print(pd.Series(ard.coef_, exog.columns.values))

statsmodels.graphics.tsaplots.plot_pacf(resid)

line1 = alt.Chart(data_df).mark_line().encode(x='dates:T', y=alt.Y('energy_cpi', title="Energy CPI")).properties(width="container")
line2 = alt.Chart(data_df).mark_line(color='red').encode(x='dates:T', y=alt.Y('fitted')).properties(width="container")
line3 = alt.Chart(data_df).mark_line(color='red').encode(x='dates:T', y=alt.Y('resid')).properties(width="container")
line1 + line2 | line3

In sample Rsq of the model is - 0.72
Out Sample Rsq of the model is - 0.79
Coefs Selected 
S&P Industrial                0.000000
Bloomberg Commd               0.000000
FIBER Steel                   0.026393
CRB Spot Commod               0.105283
S&P Energy Spot              -0.111958
FIBER US Commd                0.000000
US Avg Gasoline               0.540573
US electricity all sectors    0.199568
US Natural Gas Spot          -0.010388
US Brent Spot Avg             0.004386
FIBER Petroleum               0.036934
FIBER Metals                 -0.006454
FIBER CrudOil                 0.000000
S&P GSCI                      0.001098
dtype: float64


alt.HConcatChart(...)

In [132]:
import pmdarima as pm
from pmdarima import model_selection

modl = pm.auto_arima(resid, start_p=1, start_q=1, start_P=1, start_Q=1,
                     max_p=5, max_q=5, max_P=5, max_Q=5, seasonal=True,
                     stepwise=True, suppress_warnings=True, D=10, max_D=10,
                     error_action='ignore')

#### Lasso CV

In [230]:
from sklearn.linear_model import LassoCV, LassoLarsCV

lassocv = LassoCV(cv=5, random_state=0).fit(exog, endog)

print("Insample score (Rsq): %s" % lassocv.score(exog, endog))
print("Insample score (Rsq): %s" % lassocv.score(exog_out, outsample))

print("Coefs Selected ")
print(pd.Series(lassocv.coef_, exog.columns.values))

y_lasso = lassocv.predict(exog.values)
resid = endog.values - y_lasso

data_df = pd.DataFrame(dict(dates=endog.index, 
                            energy_cpi=endog, 
                            fitted=pd.Series(y_lasso, endog.index), 
                            resid=pd.Series(resid, endog.index)))

statsmodels.graphics.tsaplots.plot_pacf(resid)

line1 = alt.Chart(data_df).mark_line().encode(x='dates:T', y=alt.Y('energy_cpi', title="Energy CPI")).properties(width="container")
line2 = alt.Chart(data_df).mark_line(color='red').encode(x='dates:T', y=alt.Y('fitted')).properties(width="container")
line3 = alt.Chart(data_df).mark_line(color='red').encode(x='dates:T', y=alt.Y('resid')).properties(width="container")
line1 + line2 | line3

Insample score (Rsq): 0.7403592197277638
Insample score (Rsq): 0.7563107720595983
Coefs Selected 
S&P Industrial                0.031090
Bloomberg Commd              -0.009719
FIBER Steel                   0.050922
CRB Spot Commod               0.086035
S&P Energy Spot              -0.295732
FIBER US Commd                0.000000
US Avg Gasoline               0.495803
US electricity all sectors    0.258684
US Natural Gas Spot          -0.011462
US Brent Spot Avg             0.024059
FIBER Petroleum               0.058839
FIBER Metals                 -0.109722
FIBER CrudOil                 0.005222
S&P GSCI                      0.280049
dtype: float64


alt.HConcatChart(...)

In [244]:
# Adaptive Lasso 

from sklearn.linear_model import Lasso

alpha = lassocv.alpha_

g = lambda w: np.sqrt(np.abs(w))
gprime = lambda w: 1. / (2. * np.sqrt(np.abs(w)) + np.finfo(float).eps)

# Or another option:
# ll = 0.01
# g = lambda w: np.log(ll + np.abs(w))
# gprime = lambda w: 1. / (ll + np.abs(w))

n_samples, n_features = exog.shape
p_obj = lambda w: 1. / (2 * n_samples) * np.sum((endog - np.dot(exog, w)) ** 2) \
                  + alpha * np.sum(g(w))

weights = np.ones(n_features)
n_lasso_iterations = 5

for k in range(n_lasso_iterations):
    X_w = exog / weights[np.newaxis, :]
    clf = Lasso(alpha=alpha, fit_intercept=False)
    clf.fit(X_w, endog)
    coef_ = clf.coef_ / weights
    weights = gprime(coef_)
    print(p_obj(coef_))  # should go down
    
print(pd.Series(clf.coef_, exog.columns.values))

0.0001240093723842503
0.00012352894850655054
0.0001234584118279751
0.00012340884432403252
0.00012330280708110336
S&P Industrial                0.000000
Bloomberg Commd              -0.000000
FIBER Steel                   0.106553
CRB Spot Commod               0.149422
S&P Energy Spot              -0.265095
FIBER US Commd                0.000000
US Avg Gasoline               0.350196
US electricity all sectors    0.252904
US Natural Gas Spot          -0.052624
US Brent Spot Avg             0.081081
FIBER Petroleum               0.121780
FIBER Metals                 -0.136600
FIBER CrudOil                 0.000000
S&P GSCI                      0.257055
dtype: float64


#### Stacking Regression

In [231]:
X = pd.DataFrame(dict(LASSO=pd.Series(y_lasso-y_ard, endog.index), ARD=pd.Series(y_ard, endog.index)))
y = endog

y_lasso_out = lassocv.predict(exog_out.values)
y_ard_out = ard.predict(exog_out.values)
X_out = pd.DataFrame(dict(LASSO=pd.Series(y_lasso_out-y_ard_out, outsample.index), ARD=pd.Series(y_ard_out, outsample.index)))
y_out = outsample

In [232]:
from sklearn.linear_model import LinearRegression

ridgecv = LinearRegression().fit(X, y)

print("Insample score (Rsq): %s" % ridgecv.score(X, y))
print("Outsample score (Rsq): %s" % ridgecv.score(X_out, y_out))

y_ridge = ridgecv.predict(X)
resid = y.values - y_ridge

data_df = pd.DataFrame(dict(dates=y.index, 
                            energy_cpi=y, 
                            fitted=pd.Series(y_ridge, y.index), 
                            resid=pd.Series(resid, y.index)))

statsmodels.graphics.tsaplots.plot_pacf(resid)

line1 = alt.Chart(data_df).mark_line().encode(x='dates:T', y=alt.Y('energy_cpi', title="Energy CPI")).properties(width="container")
line2 = alt.Chart(data_df).mark_line(color='red').encode(x='dates:T', y=alt.Y('fitted')).properties(width="container")
line3 = alt.Chart(data_df).mark_line(color='red').encode(x='dates:T', y=alt.Y('resid')).properties(width="container")
line1 + line2 | line3

Insample score (Rsq): 0.7406312842891172
Outsample score (Rsq): 0.7517415294084934


alt.HConcatChart(...)

### Model and Monitor Process

In [144]:
dataset

{'Bloomberg Commd': 'BCOM Index',
 'FIBER Steel': 'FJPPSTL Index',
 'CRB Spot Commod': 'CRB CMDT Index',
 'FIBER US Commd': 'FIBEINDX Index',
 'FIBER Petroleum': 'FIBEPTRO Index',
 'FIBER Metals': 'FIBEMETL Index',
 'FIBER CrudOil': 'FJPPPETR Index',
 'S&P GSCI': 'SPGSCI Index',
 'S&P Industrial': 'SPGSIN Index',
 'S&P Energy Spot': 'SPGSEN Index',
 'US Avg Gasoline': 'AUTMUSAG Index',
 'US electricity all sectors': 'ST27AU Index',
 'US Natural Gas Spot': 'NGUSAVER Index',
 'US Brent Spot Avg': 'ST13SA Index'}

In [145]:
from tia.bbg import LocalTerminal

index_freq_dict = dict()
for k in dataset:
    tick = dataset[k]
    v = LocalTerminal.get_reference_data(tick, "INDX_FREQ", ignore_field_error=True).as_map()[tick]["INDX_FREQ"]
    index_freq_dict[k] = v


In [192]:
# Find the next 
last_cpi_update = cpi_api.current_data_date()
next_month_end = datetime(last_cpi_update.year, (last_cpi_update.month+2)%12, 1) - timedelta(days=1)


datetime.datetime(2022, 8, 31, 0, 0)